In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv("./data/cubic_zirconia.csv")

In [3]:
data.drop(labels=["Unnamed: 0"],axis=1,inplace=True)

In [4]:
data

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.30,Ideal,E,SI1,62.1,58.0,4.27,4.29,2.66,499
1,0.33,Premium,G,IF,60.8,58.0,4.42,4.46,2.70,984
2,0.90,Very Good,E,VVS2,62.2,60.0,6.04,6.12,3.78,6289
3,0.42,Ideal,F,VS1,61.6,56.0,4.82,4.80,2.96,1082
4,0.31,Ideal,F,VVS1,60.4,59.0,4.35,4.43,2.65,779
...,...,...,...,...,...,...,...,...,...,...
26962,1.11,Premium,G,SI1,62.3,58.0,6.61,6.52,4.09,5408
26963,0.33,Ideal,H,IF,61.9,55.0,4.44,4.42,2.74,1114
26964,0.51,Premium,E,VS2,61.7,58.0,5.12,5.15,3.17,1656
26965,0.27,Very Good,F,VVS2,61.8,56.0,4.19,4.20,2.60,682


In [5]:
x=data.drop(labels=["price"],axis=1)

In [6]:
y=data[["price"]]

In [33]:
categorical_cols=x.select_dtypes(include='object').columns

In [34]:
numerical_cols=x.select_dtypes(exclude='object').columns

In [44]:
## define the custom ranking for the each ordinal variable
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [45]:
from sklearn.impute import SimpleImputer ##handling missing values
from sklearn.preprocessing import StandardScaler #handling feature scaling
from sklearn.preprocessing import OrdinalEncoder ##ordinal encoding
##pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [47]:
##pipeline for feature engineering
num_pipeline=Pipeline(
    
    
    steps=[
        
        ('imputer',SimpleImputer()),
        ('scaler', StandardScaler())
    ]
    
    
)
    

In [48]:
cat_pipeline=Pipeline(
    
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
    
)

In [49]:
preprocessor=ColumnTransformer(
    [
        
        ('num_pipeline',num_pipeline,numerical_cols),
        ('cat_pipeline',cat_pipeline,categorical_cols)
    ]
)

In [50]:
## Train test split

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=30)

In [51]:
preprocessor.fit_transform(x_train)

array([[ 0.56167397,  0.31982286, -0.6510053 , ...,  4.        ,
         3.        ,  4.        ],
       [-0.60408689, -0.39681436,  1.58987791, ...,  2.        ,
         1.        ,  2.        ],
       [-0.52081826, -0.25348692, -1.09918195, ...,  4.        ,
         4.        ,  1.        ],
       ...,
       [ 1.87315495,  0.7498052 , -0.20282866, ...,  2.        ,
         4.        ,  4.        ],
       [-1.04124722,  1.03646009, -0.20282866, ...,  1.        ,
         0.        ,  1.        ],
       [-0.54163542, -0.68346925, -0.20282866, ...,  4.        ,
         0.        ,  2.        ]])

In [52]:
preprocessor.transform(x_test)

array([[-8.33075636e-01,  1.76495416e-01,  1.58987791e+00, ...,
         3.00000000e+00,  1.00000000e+00,  2.00000000e+00],
       [ 2.91050915e-01, -9.70124145e-01,  6.93524626e-01, ...,
         3.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 6.24125449e-01, -3.96814364e-01, -2.02828660e-01, ...,
         3.00000000e+00,  6.00000000e+00,  3.00000000e+00],
       ...,
       [-1.04124722e+00,  8.21468919e-01, -2.02828660e-01, ...,
         2.00000000e+00,  2.00000000e+00,  3.00000000e+00],
       [ 4.36771024e-01, -1.81823197e-01,  6.93524626e-01, ...,
         3.00000000e+00,  3.00000000e+00,  2.00000000e+00],
       [ 4.15953865e-01, -1.01840275e-14,  1.14170127e+00, ...,
         3.00000000e+00,  1.00000000e+00,  1.00000000e+00]])

In [53]:
preprocessor.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [54]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [55]:
x_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.561674,0.319823,-0.651005,0.704838,0.692859,0.723432,4.0,3.0,4.0
1,-0.604087,-0.396814,1.589878,-0.533052,-0.486169,-0.533329,2.0,1.0,2.0
2,-0.520818,-0.253487,-1.099182,-0.409263,-0.376688,-0.410385,4.0,4.0,1.0
3,-0.999613,0.248159,-1.099182,-1.178523,-1.134635,-1.107067,4.0,4.0,4.0
4,0.228599,0.964796,-0.651005,0.413049,0.330729,0.477544,2.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...
18871,-0.187744,-2.045080,2.038055,0.103577,0.044393,-0.150837,3.0,3.0,2.0
18872,0.436771,0.319823,0.245348,0.607575,0.558113,0.614148,3.0,3.0,2.0
18873,1.873155,0.749805,-0.202829,1.535992,1.543444,1.611360,2.0,4.0,4.0
18874,-1.041247,1.036460,-0.202829,-1.346523,-1.260959,-1.189030,1.0,0.0,1.0


In [56]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [57]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [58]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [59]:
trained_model_list=[]
model_list=[]
r2_list=[]

In [60]:
list(models)

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [61]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    print(model)

LinearRegression()
Lasso()
Ridge()
ElasticNet()


In [62]:
models.keys()

dict_keys(['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet'])

In [63]:
models.values()

dict_values([LinearRegression(), Lasso(), Ridge(), ElasticNet()])

In [64]:
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    #Make Predictions
    y_pred=model.predict(x_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1179.6361399641773
MAE: 813.7992475074371
R2 score 91.4418612352779


Lasso
Model Training Performance
RMSE: 1180.8285468233357
MAE: 815.5416840351337
R2 score 91.42455091279984


Ridge
Model Training Performance
RMSE: 1179.8152041843202
MAE: 814.073336243402
R2 score 91.43926285311181


Elasticnet
Model Training Performance
RMSE: 1597.6630046716778
MAE: 1070.6871494397335
R2 score 84.30166679563746




In [65]:
trained_model_list

[]

In [66]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']

In [67]:
r2_list

[0.914418612352779, 0.9142455091279985, 0.9143926285311181, 0.8430166679563745]